# 单词级别的One-Hot编码

## 1. 概述

One-Hot编码是自然语言处理中最基础的文本向量化方法。它将每个单词映射到一个高维稀疏向量，向量维度等于词汇表大小，只有对应单词的位置为1，其余位置都为0。

### 核心特点：
- **稀疏性**：向量中只有一个位置为1，其余全为0
- **高维度**：维度等于词汇表大小，通常为几千到几万
- **无序性**：不同单词之间没有相似度关系（正交向量）
- **离散性**：每个单词被视为独立的符号

### 优势：
- 实现简单直观
- 适合小规模文本处理
- 可解释性强

### 劣势：
- 无法捕捉语义相似性（如"猫"和"狗"的向量完全正交）
- 维度灾难（词汇表越大，向量越稀疏）
- 内存占用大
- 无法处理未登录词（OOV, Out-of-Vocabulary）

## 2. 方法一：手动实现One-Hot编码

### 实现步骤：
1. 构建词汇表：为每个唯一单词分配一个整数索引
2. 创建零矩阵：形状为(样本数, 最大序列长度, 词汇表大小)
3. 填充矩阵：将对应位置设置为1

In [ ]:
import numpy as np

# 设置随机种子以确保结果可复现
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# 示例文本数据
samples = ["The cat sat on the mat.", "The dog ate my homework."]

# 步骤1：构建词汇表
# 为每个唯一单词分配一个整数索引（从1开始，0保留用于填充）
token_index = {}
for sample in samples:
    for word in sample.split():
        if word not in token_index:
            # 索引从1开始，0通常保留用于填充或特殊标记
            token_index[word] = len(token_index) + 1

print("词汇表索引映射：")
for word, idx in sorted(token_index.items(), key=lambda x: x[1]):
    print(f"  {word:12s} -> {idx}")
print(f"\n词汇表大小: {len(token_index)}")

# 步骤2：设置序列最大长度并创建零矩阵
max_length = 10  # 只考虑每个样本的前10个单词
vocab_size = max(token_index.values()) + 1  # +1是因为索引0保留

# 创建三维零矩阵：(样本数, 序列长度, 词汇表大小)
results = np.zeros(shape=(len(samples), max_length, vocab_size))

print(f"结果矩阵形状: {results.shape}")
print(f"  - {results.shape[0]} 个样本")
print(f"  - 每个样本最多 {results.shape[1]} 个单词")
print(f"  - 每个单词编码为 {results.shape[2]} 维向量\n")

# 步骤3：填充One-Hot向量
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = token_index.get(word)
        # 将对应位置设置为1
        results[i, j, index] = 1

# 输出结果分析
print("="*60)
print("第一个样本的One-Hot编码结果：")
print("="*60)
for i, word in enumerate(samples[0].split()[:max_length]):
    word_vector = results[0, i, :]
    non_zero_idx = np.where(word_vector == 1)[0]
    print(f"位置 {i}: '{word:12s}' -> 索引 {non_zero_idx[0] if len(non_zero_idx) > 0 else 'None'}")

print(f"\n完整的One-Hot矩阵（第一个样本）：")
print(results[0])

## 3. 方法二：使用Keras Tokenizer实现

Keras提供了`Tokenizer`类来简化文本预处理流程。

### Tokenizer的核心功能：
- **fit_on_texts()**: 根据文本构建词汇表
- **texts_to_sequences()**: 将文本转换为整数序列
- **texts_to_matrix()**: 直接生成One-Hot矩阵

### 参数说明：
- **num_words**: 只保留最常见的N个单词
- **mode**: 编码模式（binary, count, tfidf, freq）

In [ ]:
from keras.preprocessing.text import Tokenizer

samples = ["The cat sat on the mat.", "The dog ate my homework."]

# 创建Tokenizer实例
# num_words参数指定只考虑最常见的N个单词（按词频排序）
tokenizer = Tokenizer(num_words=1000)

# 根据文本数据构建词汇表
tokenizer.fit_on_texts(samples)

# 将文本转换为整数序列
# 例如: "The cat sat" -> [1, 2, 3]
sequences = tokenizer.texts_to_sequences(samples)
print("整数序列表示：")
for i, (sample, seq) in enumerate(zip(samples, sequences)):
    print(f"  样本{i+1}: {sample}")
    print(f"  序列: {seq}\n")

# 直接生成One-Hot矩阵
# mode='binary': 使用二进制编码（0或1）
one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary')

print(f"One-Hot矩阵形状: {one_hot_results.shape}")
print(f"  - {one_hot_results.shape[0]} 个样本")
print(f"  - 每个样本表示为 {one_hot_results.shape[1]} 维向量\n")

# 获取词汇表
word_index = tokenizer.word_index
print(f"词汇表大小: {len(word_index)}")
print("\n词汇表索引映射：")
for word, idx in sorted(word_index.items(), key=lambda x: x[1]):
    print(f"  {word:12s} -> {idx}")

print("\nOne-Hot矩阵内容：")
print(one_hot_results)

## 4. 方法三：散列技巧（Hashing Trick）

### 散列技巧的原理：
散列技巧通过哈希函数将单词映射到固定大小的向量空间，无需显式构建词汇表。

### 核心思想：
```
index = hash(word) % dimensionality
```

### 优势：
1. **内存高效**：不需要存储完整的词汇表
2. **可扩展性强**：可以处理任意大小的词汇表
3. **支持在线学习**：新词可以直接编码
4. **实现简单**：只需要一个哈希函数

### 劣势：
1. **哈希碰撞**：不同单词可能映射到同一索引
2. **不可逆**：无法从索引还原单词
3. **碰撞率**：维度越小，碰撞概率越高

### 碰撞概率估算：
根据生日悖论，当词汇表大小为V，哈希空间维度为D时：
- 碰撞概率 ≈ 1 - exp(-V²/(2D))
- 建议：D ≥ 2V 以保持较低碰撞率

In [ ]:
import numpy as np

samples = ["The cat sat on the mat.", "The dog ate my homework."]

# 设置哈希空间维度
# 维度越大，碰撞概率越低，但内存占用也越大
dimensionality = 1000
max_length = 10

# 创建零矩阵
results = np.zeros((len(samples), max_length, dimensionality))

# 记录哈希映射以检测碰撞
hash_mapping = {}

print("散列编码过程：")
print("="*60)

for i, sample in enumerate(samples):
    print(f"\n样本 {i+1}: {sample}")
    for j, word in list(enumerate(sample.split()))[:max_length]:
        # 计算哈希值并映射到[0, dimensionality)范围
        # abs()确保哈希值为正数
        index = abs(hash(word)) % dimensionality
        
        # 记录映射关系
        if index not in hash_mapping:
            hash_mapping[index] = []
        hash_mapping[index].append(word)
        
        results[i, j, index] = 1
        print(f"  位置 {j}: '{word:12s}' -> 哈希索引 {index}")

# 检测哈希碰撞
print("\n" + "="*60)
print("哈希碰撞分析：")
print("="*60)
collisions = {idx: words for idx, words in hash_mapping.items() if len(words) > 1}

if collisions:
    print(f"发现 {len(collisions)} 个碰撞：")
    for idx, words in collisions.items():
        print(f"  索引 {idx}: {words}")
else:
    print("未发现碰撞")

unique_words = len(set(word for sample in samples for word in sample.split()))
collision_rate = len(collisions) / unique_words if unique_words > 0 else 0
print(f"\n碰撞率: {collision_rate:.2%}")
print(f"唯一单词数: {unique_words}")
print(f"哈希空间维度: {dimensionality}")
print(f"空间利用率: {unique_words/dimensionality:.2%}")

## 5. 三种方法对比

| 特性 | 手动实现 | Keras Tokenizer | 散列技巧 |
|------|----------|-----------------|----------|
| **实现难度** | 简单 | 非常简单 | 简单 |
| **内存占用** | 高 | 高 | 中 |
| **需要词汇表** | 是 | 是 | 否 |
| **处理OOV** | 不支持 | 不支持 | 支持 |
| **可解释性** | 高 | 高 | 低 |
| **碰撞风险** | 无 | 无 | 有 |
| **适用场景** | 小规模教学 | 一般NLP任务 | 大规模在线学习 |

## 6. 实际应用建议

### 选择标准：

1. **小规模任务（词汇表 < 10K）**:
   - 推荐使用 Keras Tokenizer
   - 优势：API简洁，功能完善

2. **大规模任务（词汇表 > 100K）**:
   - 考虑散列技巧或词嵌入（Word2Vec, GloVe）
   - One-Hot编码维度过高，效率低下

3. **在线学习场景**:
   - 使用散列技巧
   - 无需预先构建词汇表

4. **生产环境**:
   - 首选词嵌入方法（Embedding层、预训练模型）
   - One-Hot编码仅用于简单基线模型

### One-Hot编码的局限性：

在现代NLP中，One-Hot编码已逐渐被更高级的表示方法取代：
- **词嵌入**（Word Embeddings）：Word2Vec, GloVe, FastText
- **上下文嵌入**（Contextual Embeddings）：ELMo, BERT, GPT

这些方法能够：
1. 捕捉语义相似性
2. 降低维度（通常为100-300维）
3. 共享统计信息
4. 提升模型泛化能力